## 모델 출력 확인하기
1. 모델 학습/평가 관리용 클래스인 ModelManager의 인스턴스를 생성합니다.
2. 학습/평가용으로 불러올 전처리 데이터의 구조는 BaseDataset 클래스로 구현됩니다.
3. BaseDataset은 torch.utils.data.Dataset 클래스를 상속받는데, 이를 통해 전처리 데이터를 DataLoader로 관리할 수 있게 됩니다.
4. DataLoader는 BaseDataset에 구현된 데이터 구조를 batch data 형태로 바꾸기 위해 데이터의 모든 요소에 차원을 하나 추가합니다.<br/>
   -> 상세: BaseDataset은 인스턴스 생성 시(즉 __init__함수에서) list[dict[str, Tensor]] 형태의 데이터를 생성하고 self.behaviors_parsed에 저장합니다.<br/>
   즉 데이터를 하나 뽑으면 dict[str, Tensor] 형태의 구조를 갖습니다.<br/>
   그런데 mini batch 학습을 위해서는 여러개의 데이터를 하나로 묶어서 batch data를 생성해야 하고, 이 기능을 DataLoader로 수행합니다.<br/>
   DataLoader에서는 여러 데이터를 하나로 묶어 dict[str, list[Tensor]] 형태로 변경합니다.<br/>
   list[Tensor]의 길이는 config파일의 batch_size로 정해집니다.<br/>
   여기서 1 epoch의 총 iteration은 behaviors의 총 데이터 수 / batch_size로, 배치 데이터의 총 개수와 같습니다.<br/>
5. 모든 모델 클래스가 상속 받는 pl.LightningModule의 구현 방식으로 인해, 모델의 인스턴스 자체를 함수처럼 사용하면 해당 클래스에 구현된 forward() 함수가 실행됩니다.
6. forward 함수는 학습 시 사용하는 batch data를 받아, behaviors의 사용자 history를 기반으로 해당 사용자의 impression 목록의 click probability를 예측하고 반환합니다.<br/>
   -> 상세: behaviors의 모든 데이터는 크게 유저의 history, 해당 유저의 impressions 데이터로 구성됩니다.<br/>
   여기서 history는 해당 유저가 과거에 열람한 뉴스 목록, impressions는 이러한 history를 가진 유저에게 특정 시점에 화면에 노출된 뉴스 목록입니다.<br/>
   여기서 impressions에는 유저가 해당 뉴스를 클릭했는지(1), 하지 않았는지(0)가 1과 0으로 라벨링 되어있습니다.<br/>
   즉 모델이 history만으로 impressions의 모든 뉴스 목록에 대해 해당 history를 가진 유저의 클릭 가능성을 예측하고, 라벨과 비교하거나 순위를 매겨보면 해당 모델이 추천을 얼마나 정확하게 하는지를 계산할 수 있습니다. 
7. 여기서 반환 형태는 Tensor인데, 내부 데이터는 list[list[float]] 형태입니다. 즉 입력한 모든 batch data에 대한 예측 결과가 반환되는 것입니다.<br/>
   -> 상세: 예를 들어 batch_size가 2라면, 각 배치마다 behaviors의 데이터가 2개씩 포함될 것입니다.<br/>
    따라서 예측해야할 유저와 impression 쌍도 두개이므로, 반환하는 결과 데이터도 2개입니다.

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

## 1. ckpt 파일로 모델 불러오기

In [2]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)

from utils.model_manager import ModelManager
from utils.base_manager import ManagerArgs
from utils.news_viewer import NewsViewer

c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
args = ManagerArgs(
    config_path = path.join(PROJECT_DIR, "config/model/nrms/exp_demo1.yaml"),
    test_ckpt_path = path.join(PROJECT_DIR, "logs/lightning_logs/checkpoints/nrms/exp_demo1/epoch=24-val_auc_epoch=0.6996.ckpt")
)

model_manager = ModelManager(PROJECT_DIR, args, "test")
news_viewer = NewsViewer(
    path.join(PROJECT_DIR, "data", "MIND", "demo", "test", "news.tsv"),
    path.join(PROJECT_DIR, "data", "preprocessed_data", "demo", "test", "news2int.tsv")
)

Seed set to 1234
100%|██████████| 7538/7538 [00:05<00:00, 1279.25it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## 2. 테스트용 데이터 불러오기

In [ ]:
"""
모든 데이터의 첫 번째 차원의 shape 값은 batch size입니다.
즉 batch data 안에 실제로 어떤 데이터가 저장되어있는지 알아보기 위해 출력해볼 때
해당 값은 별로 의미가 없습니다. 

예를 들어 h_title의 shape 출력 결과의 각 숫자는 다음과 같은 의미를 지닙니다.
(batch_size, config파일에 설정한 max_history 값, 전처리 과정에서 설정한 max_title 값 = 제목의 최대 토큰 개수)

c_abstract은 다음과 같습니다.
(batch_size, 해당 impressions 데이터에 포함된 뉴스 개수, 전처리 과정에서 설정한 max_abstract 값 = 본문 요약의 최대 토큰 개수)
"""

batch_index = 0
batch_data = model_manager.get_batch_from_dataloader(batch_index)
# model_manager.show_batch_struct(batch_data)

<class 'dict'>
{
	user:	type=Tensor, shape=(1,), inner_type=int
	h_idxs:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_title:	type=Tensor, shape=(1, 50, 20), inner_type=list[list[int]]
	h_abstract:	type=Tensor, shape=(1, 50, 50), inner_type=list[list[int]]
	h_category:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_subcategory:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_vader_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	h_bert_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	history_length:	type=Tensor, shape=(1,), inner_type=int
	c_idxs:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_title:	type=Tensor, shape=(1, 28, 20), inner_type=list[list[int]]
	c_abstract:	type=Tensor, shape=(1, 28, 50), inner_type=list[list[int]]
	c_category:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_subcategory:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_vader_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	c_bert_sentime

AttributeError: 'list' object has no attribute 'tolist'

## 3. 모델에 batch data 입력하고 출력 확인하기

In [5]:
"""
index를 바꿔서 원하는 데이터를 테스트해볼 수 있습니다.
"""
result = model_manager.show_result(0)

Rank    Score    Label  index 
--------------------------------
1      21.96449    0      8   
2      21.63985    0      21  
3      14.37056    0      20  
4      13.39139    0      22  
5      7.60424     0      17  
6      4.49830     0      26  
7      3.25125     0      23  
8      2.46181     0      0   
9      0.83084     0      1   
10     0.31242     0      11  
11     0.16820     0      16  
12     -0.67941    0      27  
13     -3.86770    1      9   
14     -5.43905    0      25  
15     -6.06323    0      18  
16     -8.16474    0      24  
17     -8.50863    0      5   
18     -8.97648    0      10  
19     -9.31507    0      6   
20    -10.63651    0      15  
21    -11.07256    0      14  
22    -12.26732    0      13  
23    -12.92857    0      3   
24    -12.96733    0      12  
25    -13.71577    0      19  
26    -17.14813    0      2   
27    -24.28173    0      7   
28    -29.99214    0      4   


## 2. 추천 순위 top 5 뉴스의 정보 출력하기

In [6]:
def show_history(batch_index, sample_num):
    sample_num = max(1, sample_num)
    batch_data = model_manager.get_batch_from_dataloader(batch_index)
    news_idxs = batch_data['h_idxs'][0].tolist()
    print("================================================================")
    print(f"{sample_num} Samples of History (User: {batch_data['user'].item()})")
    print("================================================================")
    count = 0
    print("----------------------------------------------------------------")
    for news_idx in news_idxs:
        if news_idx == 0:
            continue
        print(f"[ Sample {count+1} ]")
        news_viewer.show_news_by_index(news_idx)
        print("----------------------------------------------------------------")
        count += 1
        if count >= sample_num:
            break

def show_topN_result(batch_index, topN):
    topN = max(1, topN)
    batch_data = model_manager.get_batch_from_dataloader(batch_index)
    news_idxs = batch_data['c_idxs'][0]
    print("================================================================")
    print(f"Top {topN} Impressions Ranked by Model (User: {batch_data['user'].item()})")
    print("================================================================")
    print("----------------------------------------------------------------")
    for ranking_data in result[:min(topN, len(result))]:
        rank = ranking_data['rank']
        label = ranking_data['label']
        score = ranking_data['score']
        index = ranking_data['index']
        news_idx = news_idxs[index].item()
        
        print(f"rank: {rank}, score: {score:>.5f}, label: {label}")
        news_viewer.show_news_by_index(news_idx)
        print("----------------------------------------------------------------")

In [7]:
batch_index = 1
sample_num = 5
topN = 5

model_manager.show_result(batch_index)
print("\n\n")
show_history(batch_index, sample_num)
print("\n\n")
show_topN_result(batch_index, topN)

Rank    Score    Label  index 
--------------------------------
1      16.55572    0      24  
2      13.30618    0      9   
3      12.90063    0      34  
4      9.68928     0      19  
5      9.54487     0      60  
6      9.49449     0      11  
7      9.01754     0      40  
8      8.86234     0      41  
9      8.18778     0      1   
10     8.09063     0      25  
11     7.01228     0      53  
12     5.77802     0      6   
13     5.61280     0      51  
14     5.54765     0      39  
15     5.52212     0      27  
16     5.37015     0      5   
17     5.27092     0      14  
18     4.40056     0      58  
19     4.30785     0      31  
20     3.51065     0      59  
21     2.59219     0      46  
22     1.64392     0      21  
23     0.76598     0      29  
24     0.72754     0      18  
25     -0.13379    0      15  
26     -0.20651    0      32  
27     -0.43332    0      16  
28     -0.86846    0      57  
29     -0.91166    0      7   
30     -1.64432    0      8   
31    